# Aposteriori Unimodality applications

## Real life data

### Preprocessing

In [138]:
%load_ext autoreload 
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path

import aposteriori

import re
import hashlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
df = pd.read_pickle("data/attitudes_embedded.csv")
df

,tweet,toyou,toany,annotatorMinority,bert,annotatorPolitics,traditionalism,annotatorRace,annotatorAge,annotatorGender,freeSpeech,harmHateSpeech,intent,lingPurism,racism,racist
0,""" you ugly bitch!!!""- that seems to be my real...","[2, 1, 1, 2]","[3, 3, 1, 2]","[{}, pansexual, disabled, trans, {}, {}]","[0.0047034588642418385, 0.11112377792596817, 0...","[-1.0, -1.0, 1.0, 0.5]","[3, 1, 5, 4]","[black, black, white, white]","[25.0, 35.0, 50.0, 25.0]","[man, man, woman, woman]","[1, 1, 1, 2]","[4, 5, 5, 2]","[1, 1, 1, 2]","[1, 1, 5, 5]","[1, 1, 1, 4]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[4, 2, 1, 2, 1, 1]","[4, 2, 1, 2, 2, 1]","[{}, {}, {}, {}, {}, {}]","[0.35706740617752075, 0.032704465091228485, 0....","[-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]","[4, 3, 4, 3, 1, 4]","[black, black, white, white, white, white]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[woman, woman, woman, man, man, man]","[1, 1, 3, 4, 1, 1]","[4, 4, 3, 1, 5, 4]","[4, 2, 1, 3, 1, 1]","[2, 5, 5, 5, 5, 5]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 1, 1, 1, 2]","[2, 1, 1, 1, 1, 3]","[{}, black; female, nan, {}, {}, {}]","[-0.022276882082223892, 0.17076537013053894, 0...","[-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]","[3, 4, 3, 5, 1, 3]","[black, black, white, white, white, white]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[woman, woman, man, woman, man, woman]","[1, 1, 5, 1, 1, 1]","[4, 5, 1, 5, 5, 4]","[1, 1, 1, 1, 1, 1]","[5, 4, 5, 5, 5, 4]","[1, 1, 5, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 2, 1, 1, 1, 1]","[1, 4, 1, 1, 1, 1]","[{}, black; female, nan, {}, {}, nan]","[0.07474598288536072, 0.24487990140914917, 0.2...","[0.0, -1.0, 1.0, 0.5, -0.5, -1.0]","[4, 4, 3, 4, 5, 5]","[black, black, white, white, white, white]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[man, woman, man, woman, woman, woman]","[1, 1, 5, 3, 1, 1]","[4, 5, 1, 3, 5, 5]","[1, 1, 1, 1, 1, 1]","[4, 4, 5, 5, 4, 3]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 3, 3, 4]","[1, 2, 4, 4, 4]","[{}, NONE, No i do not, none, {}]","[0.050774428993463516, 0.08187347650527954, 0....","[-1.0, 1.0, 0.5, 0.5, -1.0]","[5, 3, 5, 5, 5]","[black, black, white, white, white]","[45.0, 45.0, 45.0, 60.0, 40.0]","[man, man, woman, woman, woman]","[1, 5, 1, 1, 1]","[5, 4, 3, 5, 2]","[1, 3, 1, 3, 5]","[5, 3, 4, 5, 5]","[1, 3, 5, 1, 1]","[1, 4, 2, 2, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 2, 3]","[4, 5, 2, 3]","[pansexual, disabled, trans, {}, none, {}]","[0.22457970678806305, 0.3840716481208801, -0.4...","[-1.0, 1.0, 0.5, 0.5]","[1, 5, 5, 5]","[black, black, white, white]","[35.0, 30.0, 45.0, 65.0]","[man, woman, man, woman]","[1, 5, 1, 1]","[5, 5, 5, 5]","[1, 5, 1, 3]","[1, 5, 5, 5]","[1, 5, 1, 1]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 3, 1, 1, 1, 1]","[1, 5, 1, 1, 1, 1]","[{}, NONE, {}, {}, {}, none]","[0.1781517118215561, 0.02407737821340561, 0.13...","[-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]","[5, 3, 5, 2, 4, 3]","[black, black, white, white, white, white]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[man, man, woman, man, man, man]","[1, 5, 1, 2, 1, 1]","[5, 4, 4, 2, 3, 5]","[1, 3, 1, 1, 1, 1]","[5, 3, 5, 2, 5, 4]","[1, 5, 2, 3, 2, 1]","[1, 4, 1, 1, 1, 1]"
623,yids are not as honest as the rest of us,"[5, 5, 4, 5, 4, 1]","[5, 5, 4, 5, 4, 1]","[nan, {}, {}, none, {}, bisexual]","[0.2983386516571045, 0.31099164485931396, -0.4...","[-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]","[3, 5, 2, 5, 4, 3]","[black, black, white, white, white, white]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[man, man, man, woman, woman, woman]","[1, 1, 5, 1, 2, 1]","[3, 3, 2, 5, 3, 5]","[5, 5, 4, 5, 5, 1]","[5, 3, 5, 5, 4, 5]","[1, 1, 4, 1, 2, 1]","[5, 3, 3, 5, 5, 1]"
624,"you've got to be fucking joking, my room has l...","[3, 3, 1, 1, 2, 1]","[2, 4, 1, 2, 3, 2]","[

In [140]:
df = df.loc[:, ["tweet", "racism", "annotatorAge","annotatorRace", "annotatorGender"]]
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[25.0, 35.0, 50.0, 25.0]","[black, black, white, white]","[man, man, woman, woman]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[45.0, 45.0, 45.0, 60.0, 40.0]","[black, black, white, white, white]","[man, man, woman, woman, woman]"
...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[35.0, 30.0, 45.0, 65.0]","[black, black, white, white]","[man, woman, man, woman]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[black, black, white, white, white, white]","[man, man, woman, man, man, man]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[40.0, 25.0, 40.0, 35.0, 45.0, 35.0]","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]"


In [141]:
def extract_unique_values(df, columns):
    """
    Extract unique values from lists in the specified columns of a pandas DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to extract unique values from.
    
    Returns:
        dict: A dictionary where keys are column names and values are sets of unique values.
    """
    unique_values = {}
    for col in columns:
        unique_values[col] = set(value for sublist in df[col] for value in sublist)
    return unique_values

extract_unique_values(df, ["annotatorAge", "annotatorRace", "annotatorGender"])

{'annotatorAge': {nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  18.0,
  nan,
  25.0,
  nan,
  nan,
  nan,
  30.0,
  35.0,
  nan,
  nan,
  nan,
  40.0,
  45.0,
  nan,
  nan,
  50.0,
  nan,
  55.0,
  nan,
  60.0,
  nan,
  65.0,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan},
 'annotatorRace': {'black',
  'hisp',
  'middleEastern',
  'na',
  'native',
  'other',
  'white'},
 'annotatorGender': {'man', 'na', 'nonBinary', 'woman'}}

Clear out NaNs from dataset

In [142]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()


(41, 41, 39)

In [143]:
def find_inconsistent_rows(df):
    inconsistent_rows = []
    
    for index, row in df.iterrows():
        list_lengths = [len(row[col]) for col in df.columns if isinstance(row[col], list)]
        
        if len(set(list_lengths)) > 1:  # Check if lengths are inconsistent
            inconsistent_rows.append(index)
    
    return df.loc[inconsistent_rows]


find_inconsistent_rows(df)

,tweet,racism,annotatorAge,annotatorRace,annotatorGender


### Results

In [144]:
def extract_annotations_and_attributes(
    df: pd.DataFrame, annot_col: str, attribute_col: str, key_col: str
) -> tuple[list, list]:
    all_annotations = []
    all_attributes = []
    all_keys = []

    for _, row in df.iterrows():
        all_annotations.extend(row[annot_col])
        all_attributes.extend(row[attribute_col])
        # extend the key for each value in the above extracted list
        all_keys.extend([row[key_col]] * len(row[annot_col]))

    return all_annotations, all_attributes, all_keys


def run_aposteriori(df, column):
    annotations, attributes, keys = extract_annotations_and_attributes(
        df=df, annot_col="racism", attribute_col=column, key_col="tweet"
    )
    stat = aposteriori.aposteriori_unimodality(
        annotations=annotations, factor_group=attributes, comment_group=keys
    )
    return pd.Series(stat)

In [145]:
print(run_aposteriori(df[~nan_age], "annotatorAge"))
print(run_aposteriori(df[~nan_gender], "annotatorGender"))
print(run_aposteriori(df[~nan_race], "annotatorRace"))

25.0    2.227540e-33
35.0    8.056433e-03
50.0    1.517690e-31
18.0    1.405492e-01
30.0    2.653637e-06
40.0    1.027556e-20
45.0    1.280582e-11
55.0    3.393591e-02
60.0    2.622263e-08
65.0    1.172850e-12
dtype: float64
man          1.000000e+00
woman        1.826425e-04
nonBinary    6.034729e-08
dtype: float64
black            9.710141e-01
white            1.000000e+00
hisp             5.379128e-10
middleEastern             NaN
other            1.726807e-03
dtype: float64


/home/dimits/anaconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/dimits/anaconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


## Synthetic data

### Preprocessing

In [146]:
import re

import pandas as pd
import numpy as np


def format_dataset(df: pd.DataFrame, min_message_len: int) -> pd.DataFrame:
    df = df.astype(str)

    # Extract all annotations from the 'annotation' column
    annotations = df["annotation"].apply(get_annotations)

    # Convert each annotation dictionary into separate columns
    annotations_df = pd.json_normalize(annotations)

    # Concatenate the new columns with the original dataframe
    df = pd.concat([df, annotations_df], axis=1)
    df = df[(df.toxicity != -1) | (df.argumentquality != -1)]

    df.message_order = df.message_order.astype(int)

    # Process other columns as needed
    df.is_moderator = (df.is_moderator == "True").astype(bool)
    df["intent"] = df.user_prompt.apply(get_user_intent).astype(str)
    df.intent = np.where(df.is_moderator, "Moderator", df.intent).astype(str)

    df["not_intervened"] = (
        df.is_moderator & df.message.apply(lambda x: len(x.strip()) < min_message_len)
    ).astype(bool)

    df = df.loc[
        :,
        [
            "conv_id",
            "message_id",
            "message_order",
            "conv_variant",
            "model",
            "user",
            "user_prompt",
            "is_moderator",
            "intent",
            "message",
            "annotator_prompt"
        ]
        + list(annotations_df.columns)
        + ["not_intervened"],
    ]
    return df


def get_annotations(annot_str: str) -> dict:
    """Extracts all key-value pairs from the annotation string into a dictionary."""
    try:
        annot_str = str(annot_str).lower()
        # Regex to match key-value pairs of the form type=value
        pattern = r"(\w+)=([-\d\.]+)"
        matches = re.findall(pattern, annot_str)
        return {
            key: float(value) if "." in value else int(value) for key, value in matches
        }
    except Exception as e:
        return {}


def get_user_intent(prompt: str) -> str:
    prompt = prompt.lower()

    if "community" in prompt:
        return "Community-oriented"
    elif "troll" in prompt:
        return "Troll"
    elif "special_instructions: ," in prompt:
        return "Neutral"
    else:
        return "Unknown"


In [147]:
GRAPH_OUTPUT_DIR = Path("../graphs")
TOXICITY_TICKS = np.arange(1, 6, 1)
YTICKS_LIST = [TOXICITY_TICKS, TOXICITY_TICKS]
VMIN_TOXICITY = -1
VMAX_TOXICITY = 1

MODERATION_STRATEGY_MAP = {
    "vanilla": "No Instructions",
    "moderation_game": "Moderation Game",
    "no_moderator": "No Moderator",
    "erulemaking": "Human Mod. Guidelines",
    "constructive_communications": "Human Fac. Guidelines",
    "collective_constitution": "Rules Only",
}

MODEL_MAP = {
    "hardcoded": "hardcoded",
    "mistral-nemo-abliterated": "Mistral Nemo (abl.)",
    "qwen2.5-32b-4bit": "Qwen 2.5",
    "llama-3.1-70b-4bit": "LLaMa 3.1",
}

METRIC_MAP = {
    "toxicity": "Toxicity",
    "argumentquality": "Argument Quality"
}
METRICS = METRIC_MAP.values()

In [148]:
full_df = pd.read_csv("data/dataset.csv", encoding="utf8")

full_df.conv_variant = full_df.conv_variant.map(MODERATION_STRATEGY_MAP)
full_df.model = full_df.model.map(MODEL_MAP)
full_df = format_dataset(full_df, min_message_len=3)
full_df = full_df.rename(METRIC_MAP, axis=1)
full_df = full_df.drop(["conv_variant", "model", "user", "is_moderator", "not_intervened"], axis=1)
full_df

,conv_id,message_id,message_order,user_prompt,intent,message,annotator_prompt,Toxicity,Argument Quality
0,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,2,3
1,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,2,3
2,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,3,2
3,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,3,2
4,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,2,3
...,...,...,...,...,...,...,...,...,...
61142,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3
61143,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3
61144,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3
61145,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3


In [149]:
def extract_key_value_pairs(text: str) -> dict:
    pattern = re.compile(r'(?P<key>\w+): (?P<value>[^,\n]+)')
    matches = pattern.findall(text)
    
    result = {}
    for key, value in matches:
        key = key.strip()
        value = value.strip()
        
        # Convert list-like strings to actual lists
        if value.startswith("[") and value.endswith("]"):
            try:
                value = ast.literal_eval(value)
            except Exception:
                pass  # Keep as string if parsing fails
        
        result[key] = value

    
    return result
    
extract_key_value_pairs(full_df.user_prompt.iloc[0])

{'traits': 'username: RebelCoder07',
 'age': '32',
 'sex': 'non-binary',
 'sexual_orientation': 'Bisexual',
 'demographic_group': 'Black',
 'current_employment': 'Cybersecurity Expert',
 'education_level': "Master's",
 'personality_characteristics': "['rebellious'",
 'instructions': 'You are a human in an online forum.'}

In [150]:
def expand_dataframe(df: pd.DataFrame, column: str) -> pd.DataFrame:
    extracted_data = df[column].apply(extract_key_value_pairs)
    extracted_df = pd.json_normalize(extracted_data)
    return extracted_df


user_df = expand_dataframe(full_df, "user_prompt")
annot_df = expand_dataframe(full_df, "annotator_prompt")
extracted_df = user_df.join(annot_df, how="inner", lsuffix="_user", rsuffix="_annot")
extracted_df

,traits_user,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,personality_characteristics_user,instructions_user,special_instructions,...,current_employment_annot,education_level_annot,personality_characteristics_annot,instructions_annot,1,2,3,4,5,posted
0,username: RebelCoder07,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's,['rebellious',You are a human in an online forum.,NaN,...,Farmer,No formal education,['hardworking',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
1,username: RebelCoder07,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's,['rebellious',You are a human in an online forum.,NaN,...,Retired Philosopher,PhD,['wise',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
2,username: RebelCoder07,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's,['rebellious',You are a human in an online forum.,NaN,...,Botanist,Master's,['calm',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
3,username: RebelCoder07,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's,['rebellious',You are a human in an online forum.,NaN,...,Cybersecurity Expert,Master's,['rebellious',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
4,username: RebelCoder07,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's,['rebellious',You are a human in an online forum.,NaN,...,Historian,PhD,['analytical',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61142,just,NaN,NaN,NaN,NaN,NaN,NaN,NaN,You are a moderator in an online forum.,NaN,...,Travel Blogger,Some College,['spontaneous',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
61143,just,NaN,NaN,NaN,NaN,NaN,NaN,NaN,You are a moderator in an online forum.,NaN,...,Cybersecurity Expert,Master's,['rebellious',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
61144,just,NaN,NaN,NaN,NaN,NaN,NaN,NaN,You are a moderator in an online forum.,NaN,...,Registered Nurse,Bachelor's,['compassionate',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88
61145,just,NaN,NaN,NaN,NaN,NaN,NaN,NaN,You are a moderator in an online forum.,NaN,...,Poet,Master's,['sensitive',You will be given a series of comments made on...,Not Toxic,Barely Toxic,Almost Toxic,Toxic,Extremely Toxic,@TechGuru88


In [151]:
extracted_df2 = extracted_df.drop([
    "personality_characteristics_user", 
    "personality_characteristics_annot", 
    "instructions_user", 
    "instructions_annot",
    "traits_user",
    "traits_annot",
    "special_instructions",
    "moderator",
    "ban",
    "intervention",
    "posted",
    "polarization",
    "guidelines",
    "participant",
    "1", "2", "3", "4", "5"],
    axis=1)
extracted_df2.columns

Index(['age_user', 'sex_user', 'sexual_orientation_user',
       'demographic_group_user', 'current_employment_user',
       'education_level_user', 'age_annot', 'sex_annot',
       'sexual_orientation_annot', 'demographic_group_annot',
       'current_employment_annot', 'education_level_annot'],
      dtype='object')

In [152]:
full_df2 = full_df.join(extracted_df2, how="inner")
full_df2

,conv_id,message_id,message_order,user_prompt,intent,message,annotator_prompt,Toxicity,Argument Quality,age_user,...,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot
0,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,2,3,32,...,Bisexual,Black,Cybersecurity Expert,Master's,50,male,Heterosexual,Middle Eastern,Farmer,No formal education
1,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,2,3,32,...,Bisexual,Black,Cybersecurity Expert,Master's,65,male,Heterosexual,South Asian,Retired Philosopher,PhD
2,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,3,2,32,...,Bisexual,Black,Cybersecurity Expert,Master's,45,female,Heterosexual,Indigenous,Botanist,Master's
3,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,3,2,32,...,Bisexual,Black,Cybersecurity Expert,Master's,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's
4,b7674ce3-6caf-443a-b076-b6ae35f421a2,741081141450932242,1,You are a human participating in an online cha...,Neutral,"In other words, he’s not brown enough for ya t...",You are a human working as an annotator Your n...,2,3,32,...,Bisexual,Black,Cybersecurity Expert,Master's,60,male,Asexual,Caucasian,Historian,PhD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61142,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3,NaN,...,NaN,NaN,NaN,NaN,40,female,Heterosexual,Pacific Islander,Travel Blogger,Some College
61143,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3,NaN,...,NaN,NaN,NaN,NaN,32,non-binary,Bisexual,Black,Cybersecurity Expert,Master's
61144,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3,NaN,...,NaN,NaN,NaN,NaN,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's
61145,c8c3eda7-92ec-4880-a6cf-4b26bb1f3815,1928277485472945723,29,You are a human participating in an online cha...,Moderator,"@AdventureSeeker34, thank you for clarifying y...",You are a human working as an annotator Your n...,1,3,NaN,...,NaN,NaN,NaN,NaN,27,female,Lesbian,Arab,Poet,Master's


In [153]:
final_df = full_df2[~(full_df2.message == "nan")]

In [154]:
final_df = final_df.groupby(["conv_id", "message"]).agg(list)
final_df = final_df.drop(["message_order", "user_prompt", "annotator_prompt", "message_id"], axis=1)
final_df

intent  \
conv_id                              message                                                                                                 
028f869d-cf12-444a-aefc-62f8b6294acd @FitnessJunkie07 Respect and honesty are univer...  [Neutral, Neutral, Neutral, Neutral, Neutral, ...   
                                     @FitnessJunkie07, I agree, but sometimes change...  [Neutral, Neutral, Neutral, Neutral, Neutral, ...   
                                     @FitnessJunkie07, I completely agree that respe...  [Community-oriented, Community-oriented, Commu...   
                                     @FitnessJunkie07, I understand where you're com...  [Community-oriented, Community-oriented, Commu...   
                                     @LoyalVolunteer54 @FitnessJunkie07 It's importa...  [Moderator, Moderator, Moderator, Moderator, M...   
...                                                                                                                                    ...   
fe858614-6571-43e7-badf-3cdcb38337b3 @OptimisticTeacher59, you're cherry-picking sta...  [Neutral, Neutral, Neutral, Neutral, Neutral, ...   
                                     @OptimisticTeacher59, you're just as bad as @Ad...  [Troll, Troll, Troll, Troll, Troll, Troll, Troll]   
                                     @StoicSamurai99, you're missing the point. It's...  [Neutral, Neutral, Neutral, Neutral, Neutral, ...   
                                     @StoicScholar73, thank you for your kind words!...  [Neutral, Neutral, Neutral, Neutral, Neutral, ...   
                                     Nothing in the Bible about Jesus saying anythin...  [Neutral, Neutral, Neutral, Neutral, Neutral, ...   

                                                                                                                  Toxicity  \
conv_id                              message                                                                                 
028f869d-cf12-444a-aefc-62f8b6294acd @FitnessJunkie07 Respect and honesty are univer...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
                                     @FitnessJunkie07, I agree, but sometimes change...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
                                     @FitnessJunkie07, I completely agree that respe...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
                                     @FitnessJunkie07, I understand where you're com...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
                                     @LoyalVolunteer54 @FitnessJunkie07 It's importa...  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]   
...                                                                                                                    ...   
fe858614-6571-43e7-badf-3cdcb38337b3 @OptimisticTeacher59, you're cherry-picking sta...              [1, 1, 1, 1, 1, 1, 1]   
                                     @OptimisticTeacher59, you're just as bad as @Ad...              [4, 4, 4, 4, 4, 4, 4]   
                                     @StoicSamurai99, you're missing the point. It's...              [2, 2, 2, 2, 2, 2, 2]   
                                     @StoicScholar73, thank you for your kind words!...              [1, 1, 1, 1, 1, 1, 1]   
                                     Nothing in the Bible about Jesus saying anythin...              [2, 2, 2, 2, 2, 2, 2]   

                                                                                                          Argument Quality  \
conv_id                              message                                                                                 
028f869d-cf12-444a-aefc-62f8b6294acd @FitnessJunkie07 Respect and honesty are univer...  [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]   
                                     @FitnessJunkie07, I agree, but sometimes change...  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]   
                                     @FitnessJunkie07, I completely agree that respe...  [3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3]   
                

In [155]:
final_df.to_csv("data/synthetic.csv")

In [156]:
final_df.iloc[18]

intent                      [Neutral, Neutral, Neutral, Neutral, Neutral, ...
Toxicity                                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Argument Quality                            [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
age_user                         [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
sex_user                    [male, male, male, male, male, male, male, mal...
sexual_orientation_user     [Heterosexual, Heterosexual, Heterosexual, Het...
demographic_group_user      [Middle Eastern, Middle Eastern, Middle Easter...
current_employment_user     [Farmer, Farmer, Farmer, Farmer, Farmer, Farme...
education_level_user        [No formal education, No formal education, No ...
age_annot                        [38, 45, 27, 60, 32, 55, 28, 21, 50, 65, 40]
sex_annot                   [female, female, female, male, non-binary, mal...
sexual_orientation_annot    [Heterosexual, Heterosexual, Lesbian, Asexual,...
demographic_group_annot     [Latino, Indigenous, Arab, Caucasian